<a href="https://colab.research.google.com/github/joaochenriques/2021-code-smells/blob/main/Suspensao_V02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sympy import *
from sympy.printing.lambdarepr import lambdarepr
init_printing() 
from IPython.display import Math
import scipy.integrate
import numpy as np

In [ ]:
t, ξ_0, ξ_1, ξ_2 = symbols( r"t, \xi_0, \xi_1, \xi_2" )
z_F1, z_F2, z_41, y_F1, y_F2 = symbols( r"z_{F1}, z_{F2}, z_{41}, y_{F1}, y_{F2}" )
h, h_z, ℓ_4, h_4, k_4, M, g, a_z, θ_0 = symbols( r"h, h_z, \ell_4, h_4, k_4, M, g, a_z, \theta_0" )
δ_y1, δ_z1, δ_y2, δ_z2, δ_ZM, δ_YM, θ = symbols( r"\delta_{y1}, \delta_{z1}, \delta_{y2}, \delta_{z2}, \delta_{ZM}, \delta_{YM}, \theta" )
F, ΔF, F_1, F_2, F_1y, F_1z, F_2y, F_2z, y_Fz, z_Fz = symbols( r"F, \Delta{F}, F_1, F_2, F_{1y}, F_{1z}, F_{2y}, F_{2z}, y_{Fz}, z_{Fz}" )
F_z = symbols( r"F_z" )
RHS_1, RHS_2, RHS_3 = symbols( r"RHS_1, RHS_2, RHS_3" )

In [ ]:
deg2rad = pi / 180

reps = { 
    ℓ_4: 0.7,
    h_4: 0.5,
    ξ_0: 0.5,
    k_4: 1E4,
    θ_0: 20*deg2rad,
    h: 1.2,
    h_z: 0.5,
    M: 13000/2,
    g: 9.8,
    a_z: 2,
    F_z: 500,
    #
    ξ_1: sqrt( (z_F1 + ℓ_4)**2 + (y_F1-h_4)**2 ),
    ξ_2: sqrt( (z_F2 - ℓ_4)**2 + (y_F2 - h_4)**2 ),
    #
    F_1: k_4 * (ξ_0 - ξ_1) + M * g / ( 2 * cos(θ_0) ),
    F_2: k_4 * (ξ_0 - ξ_2) + M * g / ( 2 * cos(θ_0) ),
    F: (F_1 + F_2 ) / 2,
    ΔF: F_1 - F,
    #
    z_F1: δ_ZM + (h - (h_4+ξ_0*cos(θ_0) ) )*sin(θ) - ( ℓ_4 - ξ_0*sin(θ_0) )*cos(θ),
    z_F2: δ_ZM + (h - (h_4+ξ_0*cos(θ_0) ) )*sin(θ) + ( ℓ_4 - ξ_0*sin(θ_0) )*cos(θ),
    y_F1: h + δ_YM - (h - (h_4+ξ_0*cos(θ_0) ) )*cos(θ) - ( ℓ_4 - ξ_0*sin(θ_0) )*sin(θ),
    y_F2: h + δ_YM - (h - (h_4+ξ_0*cos(θ_0) ) )*cos(θ) + ( ℓ_4 - ξ_0*sin(θ_0) )*sin(θ),
    z_Fz: δ_ZM + (h - h_z )*sin(θ),
    y_Fz: h + δ_YM - (h - h_z )*cos(θ),
    #
    F_1z: F_1 * sin(θ_0 - θ),
    F_2z: F_2 * sin(θ_0 + θ),
    F_1y: F_1 * cos(θ_0 - θ),
    F_2y: F_2 * cos(θ_0 + θ),
    #
    δ_z1: δ_ZM - z_F1,
    δ_y1: h + δ_YM - y_F1,
    δ_z2: -δ_ZM + z_F2,
    δ_y2: h + δ_YM - y_F2
}
for key, values in reps.items():
    Math( latex(key) )

In [ ]:
eq_B01 = Eq( RHS_1, F_1z - F_2z - F_z + M * a_z ).subs( reps ).subs( reps ).subs( reps )
eq_B01

In [ ]:
eq_B02 = Eq( RHS_2, F_1y + F_2y - M * g ).subs( reps ).subs( reps ).subs( reps )
eq_B02

In [ ]:
eq_B03 = Eq( RHS_3, F_1y*δ_z1 - F_1z*δ_y1 - F_2y*δ_z2 + F_2z*δ_y2 + F_z*(h - h_z)*cos(θ) ).subs( reps ).subs( reps ).subs( reps )
eq_B03

In [ ]:
func = lambdify( [t,δ_ZM,δ_YM,θ], [eq_B01.rhs, eq_B02.rhs, eq_B03.rhs] )
U0 = [0,0,0]
t_eval = np.linspace(0, 5, 50)

In [ ]:
solution = scipy.integrate.solve_ivp( func, (0, 5), U0, t_eval=t_eval )